In [11]:
%load_ext autoreload
%autoreload 2

import optuna
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import shap
import numpy as np
import scipy
import seaborn as sns

from steps.prepare_data import load_processed_data, load_split_processed_data
from utils.model import predict, load_model, predict_booster

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
train_data, test_data = load_split_processed_data()

model_name = "LightGbmV2_drop_high_correlation.pickle"

train_data_x = train_data.drop(columns=["target"])
train_data_y = train_data["target"]

test_data_x = test_data.drop(columns=["target"])
test_data_y = test_data["target"]

print("-----------Train----------")
predict_booster(model_name, train_data)

print("-----------Test----------")
predict_booster(model_name, test_data)

model, features = load_model(model_name)

-----------Train----------

Metrics
AUC: 0.99
Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98    140413
           1       0.92      0.59      0.72      9585

    accuracy                           0.97    149998
   macro avg       0.94      0.79      0.85    149998
weighted avg       0.97      0.97      0.97    149998

-----------Test----------

Metrics
AUC: 0.90
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97    140597
           1       0.69      0.33      0.45      9403

    accuracy                           0.95    150000
   macro avg       0.82      0.66      0.71    150000
weighted avg       0.94      0.95      0.94    150000



In [7]:
import lightgbm as lgb
import sklearn
import eli5
from IPython.display import display
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import StratifiedKFold

# Parameters for the LGBMClassifier
params = {
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "scale_pos_weight": 1.0,
    "lambda_l1": 0.04517778689409139,
    "lambda_l2": 9.695998315685959,
    "learning_rate": 0.0981671869084374,
    "num_leaves": 182,
    "feature_fraction": 0.9536349519282109,
    "bagging_fraction": 0.6198833174062659,
    "max_depth": 45,
    "min_child_samples": 37,
}

# Stratified K-Folds cross-validator
fold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

# Train and evaluate the model
for fold_, (trn_idx, val_idx) in enumerate(fold.split(train, target)):
    print(f"%%%%%%%%%%%%%% Fold idx: {fold_ + 1} %%%%%%%%%%%%%%%")

    trn, trg = train.iloc[trn_idx], target[trn_idx]
    trn_data = lgb.Dataset(trn, label=trg)
    val_data = lgb.Dataset(train.iloc[val_idx], label=target[val_idx])

    clf = lgb.LGBMClassifier(**params, n_estimators=1000, silent=-1, verbose=-1)
    clf.fit(train.iloc[trn_idx], target[trn_idx], verbose=100)
    print("Trained")

    perm = PermutationImportance(clf, scoring=None, n_iter=1, random_state=42, cv=None, refit=False).fit(
        train.iloc[val_idx], target[val_idx]
    )
    display(eli5.show_weights(perm, top=len(list(train.columns)), feature_names=list(train.columns)))

ImportError: cannot import name 'if_delegate_has_method' from 'sklearn.utils.metaestimators' (/Users/yaroslav/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/utils/metaestimators.py)

In [8]:
import sklearn

print(sklearn.__version__)

1.4.1.post1
